## 0. Import lib and param

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sklearn
import scipy.sparse 

for p in [np, pd, sklearn, scipy]:
    print (p.__name__, p.__version__)

numpy 1.21.4
pandas 1.3.4
sklearn 1.0.1
scipy 1.7.2


In [3]:
##load dataset
file = "data/processed_data"

train_info_path = file + "/train_info.csv"
train_work_path = file + "/train_work.csv"
train_label_path = "data/label_train.csv"

test_info_path = file + "/test_info.csv"
test_work_path = file + "/test_work.csv"
test_label_path =  "data/label_test.csv"


## data frame
train_info = pd.read_csv(train_info_path)
train_work = pd.read_csv(train_work_path)
train_label = pd.read_csv(train_label_path)

test_info = pd.read_csv(test_info_path)
test_work = pd.read_csv(test_work_path)
test_label = pd.read_csv(test_label_path)

In [4]:
train_info.shape, train_work.shape, train_label.shape

((27502, 5), (247559, 19), (27502, 2))

In [5]:
# info: id_bh,	bithYear,	gender,	new_province_id
# label: id_bd, label
# work: main:id_bh , new_work_province_id ,  employee_lv , 
## year_from_date , year_to_date ,  month_from_date, month_to_date 
## num_year_contract, num_month_contract   
## company_type,   id_management ,id_office ,  job_role_encode_knn                                                                                       

## cach xu ly:
## 1. group by thep id_bh o work 
## 2. merge voi info va label

## 1. Segment text - job/role filled None (in train_work)

In [6]:
train_work.shape, test_work.shape

((247559, 19), (162283, 19))

In [7]:
train_work = pd.read_csv("data/processed_data/segment_train_work.csv")
test_work = pd.read_csv("data/processed_data/segment_test_work.csv")
train_work.shape, test_work.shape

((247559, 19), (162283, 19))

## 2. Extract text features (phoBERT)

## 3. Extract text features (TFIDF)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD

vectorizer = TfidfVectorizer(
    ngram_range = (1,2),
        min_df=5, 
    max_df= 0.8, 
    max_features=1000,
    sublinear_tf=True
)
## TF-IDF + SVD
clf = Pipeline([
                ('tfidf', vectorizer ),
                ('svd', TruncatedSVD(n_components = 128, random_state=42)),
                ])
job_role_train = train_work['job_role_fillNan'].values
job_role_test = test_work['job_role_fillNan'].values
# clf.fit_transform(job_role_train)

In [9]:
# train_job_role_tfidf = clf.transform(job_role_train)
# test_job_role_tfidf = clf.transform(job_role_test)

- Group by thanh list => chuyen thanh text khong => tf-idf

## 4. Tao DF de co the train, test

In [10]:
def concat_data(df, df_add, col='id_bh', how='left'):
    df = pd.merge(df, df_add, on=col, how=how)
    return df

In [11]:
use_cosl_train  = ["id_bh" , "new_work_province_id" ,  "employee_lv" , 
"year_from_date" , "year_to_date" ,  "month_from_date", "month_to_date" ,
"num_year_contract", "num_month_contract"   , 
# "job_n_word",
"company_type",   "id_management" ,"id_office" ,  "job_role_encode_knn", 'job_role_fillNan',
"bithYear", "gender", "new_province_id", "label"  ] 
# train
group_train_work = train_work.groupby('id_bh').agg(lambda x: x.tolist())
train = concat_data(group_train_work, train_info, col='id_bh', how='left')
train = concat_data(train, train_label, col='id_bh', how='left')
train = train[use_cosl_train]
# test
group_test_work = test_work.groupby('id_bh').agg(lambda x: x.tolist())
test = concat_data(group_test_work, test_info, col='id_bh', how='left')
test = test[use_cosl_train[:-1]]

In [12]:
def  list2str(list):
  str =""
  for i in list:
    str += (i +" ")
  return str
train['job_role_fillNan_sentence'] = train.apply(lambda row: list2str(row.job_role_fillNan), axis = 1 )
test['job_role_fillNan_sentence'] = test.apply(lambda row: list2str(row.job_role_fillNan), axis = 1 )



In [13]:
clf.fit(train['job_role_fillNan_sentence'].values)
train['job_role_TFIDF'] = list(clf.transform(train['job_role_fillNan_sentence'].values))
test['job_role_TFIDF'] = list(clf.transform(test['job_role_fillNan_sentence'].values))


In [14]:
## create col form numpy arr in jobrole tfidf
for i in range(128):
  train['job_role_TFIDF_' + str(i)] = train['job_role_TFIDF'].apply(lambda x: float(x[i]))
  test['job_role_TFIDF_' + str(i)] = test['job_role_TFIDF'].apply(lambda x: float(x[i]))

C:\Users\Toan\AppData\Local\Temp/ipykernel_1020/1102472247.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train['job_role_TFIDF_' + str(i)] = train['job_role_TFIDF'].apply(lambda x: float(x[i]))
C:\Users\Toan\AppData\Local\Temp/ipykernel_1020/1102472247.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  test['job_role_TFIDF_' + str(i)] = test['job_role_TFIDF'].apply(lambda x: float(x[i]))


In [15]:
## export 
# train.to_csv(r"data/processed_data/train_df_addTFIDFjob.csv")
# test.to_csv(r"data/processed_data/test_df_addTFIDFjob.csv")

## load lai train, test

In [16]:
# ## reload train, test
# train = pd.read_csv('data/processed_data/train_df_addTFIDFjob.csv')
# test = pd.read_csv('data/processed_data/test_df_addTFIDFjob.csv')

In [17]:
train.shape, test.shape

((27502, 148), (18134, 147))

In [18]:
train.select_dtypes(exclude=[np.number]).columns

Index(['new_work_province_id', 'employee_lv', 'year_from_date', 'year_to_date',
       'month_from_date', 'month_to_date', 'num_year_contract',
       'num_month_contract', 'company_type', 'id_management', 'id_office',
       'job_role_encode_knn', 'job_role_fillNan', 'job_role_fillNan_sentence',
       'job_role_TFIDF'],
      dtype='object')

- **NOTE: con feature job_role_TFIDF: khi group by co the dung lay mean**

## 5. Feature engineering

In [19]:
# 2. Feature engineering
## category list: company_type,	id_management,	id_office,	job_role_encode_knn ==> max frequent, last, count unique
### new_work_province_id: max frequnt, last , count unique

### new_work_province_id, id_office, company_type, id_management


## number:
### employee_lv bo gia tri -1: max, min (bo -1), mode(bo -1), std, mean, gia tri (max-min), count unique
### year_from_date: max, min, std, mode, mean, count unique
### year_to_date: max, min, std, mode, mean, count unique
### month_from_date, month_to_date:  max, min, std, mode, mean
### num_year_contract: max, min, mode, mean, std, q(75), q(25), count unique
### num_month_contract: max, min, mode, mean, std, q(75), q(25), count unique
## create new fea:



In [20]:
from scipy.stats import skew, kurtosis
data = np.random.normal(0, 1, 100)

print("mean : ", np.mean(data))
print("var  : ", np.var(data))
print("skew : ",skew(data))
print("kurt : ",kurtosis(data))

mean :  0.10379776654016885
var  :  1.160246445012187
skew :  -0.15389336944287954
kurt :  0.0769915069507845


In [21]:
from scipy.stats import skew, kurtosis
def most_frequent(List):
    return max(set(List), key = List.count)
def mean_out_nan(list):
    if (-1 in list):
        list.remove(-1)
        return sum(list) / float(len(list))
    return sum(list) / float(len(list))
def bining_olds(x):
    if (x < 0):
        return -999
    elif(0< x < 20 ):
        return 0
    elif( 20<= x < 30 ):
        return 1
    elif (30<= x < 40):
        return 2
    elif (40<= x < 50):
        return 3
    elif (50<= x < 60):
        return 4
    return 5

List = [2, 1, 2, 1, 3, - 1]
# print(count_num_jobs_outNan( List))

In [133]:
import statistics
from statistics import mode
category_cols = [
    "company_type",	"id_management",	"id_office",	"job_role_encode_knn" ,"new_work_province_id"
]
numeric_cols = [
        "employee_lv" , 
        "year_from_date",
        "year_to_date",
        "month_from_date",
         "month_to_date",
        "num_year_contract",
        "num_month_contract",
        # "job_n_word"
]
## utils func for category var
def max_frequent(list):
    if (len(list) == 0):
        return -999
    return(most_frequent(list))
def max_frequent_without_filled_var(list):
    if( -999 in list):
        list.remove(-999)
    if(-1 in list):
        list.remove(-1)
    if(-1.0 in list):
        list.remove(-1.0)
    a_set = set(list)
    if ((len(list) == 0) or (len(a_set) == len(list)) ):
        return -999
    return(most_frequent(list))
def last_value(list):
    if (len(list) == 0):
        return -999
    return list[-1]
def count_unique(list):
    return len(set(list))
def count_unique_without_filled_var(list):
    if( -999 in list):
        list.remove(-999)
    if(-1 in list):
        list.remove(-1)
    return len(set(list))

## utils fun for numeric features: max, min, mode, mean, std, q(75), q(25), count unique
def percentile_25(x):
    return x.quantile(.25)
def percentile_75(x):
    return x.quantile(.75)  
def min_without_filled_var(list):
    if(-1 in list):
        list.remove(-1)
        if (len(list) == 0):
            return -1
        else: 
            return min(list)
    return min(list)
def std(list):
    if (len(list) < 2):
            return -999
    if(-1 in list):
        list.remove(-1)
        if (len(list) < 2):
            return -999
        else: 
            return statistics.stdev(list)
    return statistics.stdev(list)
def mean_out_nan(list):
    if (-1 in list):
        list.remove(-1)
        return sum(list) / float(len(list))

    return (sum(list) / float(len(list)))
def count_num_jobs_outNan(list):
    if (-1 in list):
        list.remove(-1)
        return len(list)
    return len(list)
######################
def features_engineering(df):
    ## categorical cols
    for col in category_cols:
        df['category_max_fre_' + col] = df[col].apply(lambda x: max_frequent(x) ) ## category
        df['category_max_fre_out_filled_var_' + col] = df[col].apply(lambda x: max_frequent_without_filled_var(x) ) ## category
        df['category_last_value_' + col] = df[col].apply(lambda x: last_value(x) ) ## category
        df['numeric_count_unique_' + col] = df[col].apply(lambda x: count_unique(x) ) ## numeric
        df['numeric_count_unique_without_filled_var' + col] = df[col].apply(lambda x: count_unique_without_filled_var(x) ) ## numeric
    ## count features support cate cols:


    ## numeric cols:
    for col in numeric_cols:
        ## chi co employee_lv la co fill nan
        if (col == 'employee_lv'):
            df['numeric_max_lv_' + 'of_'+ col] = df[col].apply(lambda x: max(x) )
            df['numeric_min_lv_' + 'of_'+ col] = df[col].apply(lambda x: min_without_filled_var(x) )
            df['numeric_mode_' + 'of_'+ col] = df[col].apply(lambda x: max_frequent(x) ) 
            df['numeric_mode_out_filled_var_' + 'of_'+ col] = df[col].apply(lambda x: max_frequent_without_filled_var(x) ) 
            df['numeric_mean_' + 'of_'+ col] = df[col].apply(lambda x: sum(x) / float(len(x)) ) 
            df['numeric_mean_out_nan_' + 'of_'+ col] = df[col].apply(lambda x:  mean_out_nan(x))
            df['numeric_std_' + 'of_'+ col] = df[col].apply(lambda x: std(x) )
            df['numeric_count_unique_' + 'of_'+ col]  = df[col].apply(lambda x: count_unique(x) ) ## numeric
            df['numeric_count_unique_out_fillNan_' + 'of_'+ col]  = df[col].apply(lambda x: count_unique_without_filled_var(x) ) ## numeric
            df['numeric_Q25_' + 'of_'+ col ] = df[col].apply(lambda x: np.quantile(x, .25) )
            df['numeric_Q75_' + 'of_'+ col ] = df[col].apply(lambda x: np.quantile(x, .75) )

            df['numeric_skew_' + 'of_'+ col ] = df[col].apply(lambda x: skew(x) )
            df['numeric_kurtosis_' + 'of_'+ col ] = df[col].apply(lambda x: kurtosis(x) )
        else:
            df['numeric_max_' + 'of_'+ col] = df[col].apply(lambda x: max(x) )
            df['numeric_min_' + 'of_'+ col] = df[col].apply(lambda x: min_without_filled_var(x) )
            df['numeric_mode_' + 'of_'+ col] = df[col].apply(lambda x: max_frequent(x) )  
            df['numeric_mean_' + 'of_'+ col] = df[col].apply(lambda x: sum(x) / float(len(x)) ) 
            df['numeric_std_' + 'of_'+ col] = df[col].apply(lambda x: std(x) )
            df['numeric_count_unique_' + 'of_'+ col]  = df[col].apply(lambda x: count_unique(x) ) ## numeric
            df['numeric_Q25_' + 'of_'+ col ] = df[col].apply(lambda x: np.quantile(x, .25) )
            df['numeric_Q75_' + 'of_'+ col ] = df[col].apply(lambda x: np.quantile(x, .75) )
            df['numeric_skew_' + 'of_'+ col ] = df[col].apply(lambda x: skew(x) )
            df['numeric_kurtosis_' + 'of_'+ col ] = df[col].apply(lambda x: kurtosis(x) )
            
    ## create_new_fea between 2 features:(year_from_date, year_to_date) with bithYear
    df['numeric_year_diff_max_year_from_vs_bithYear'] = df['numeric_max_of_year_from_date'] - df['bithYear']
    df['numeric_year_diff_min_year_from_vs_bithYear'] = df['numeric_min_of_year_from_date'] - df['bithYear']
    df['numeric_year_diff_max_year_to_vs_bithYear'] = df['numeric_max_of_year_to_date'] - df['bithYear']
    df['numeric_year_diff_min_year_to_vs_bithYear'] = df['numeric_min_of_year_to_date'] - df['bithYear']

    ## old:
    df['numeric_olds'] = 2022 - df['bithYear']
    ## ratio: num_old vs diff_min_year_from_vs_bithYear
    df['numeric_ratio_olds_vs_(diff_min_year_from_vs_bithYear)'] = df['numeric_olds'] / df['numeric_year_diff_min_year_from_vs_bithYear']
    df['numeric_ratio_olds_vs_(diff_min_year_from_vs_bithYear)'] = df['numeric_ratio_olds_vs_(diff_min_year_from_vs_bithYear)'].apply(lambda x: x if(x> 0) else -999)

    ## dem so lan xuat hien trong bang: 
    df['numeric_count_job'] = df['employee_lv'].apply(lambda x: len(x))
    df['numeric_count_job_withoutNan'] = df['employee_lv'].apply(lambda x: count_num_jobs_outNan(x))

    ## fix value in create_new_fea cols
    new_cols = [col for col in df.columns if 'numeric_year_diff_' in col]
    for col in new_cols:
        df[col] = df[col].apply(lambda x: x if(x> 0) else -999)


    # new_category_cols.append('new_province_id')
      #### phase 2:
      #### category_last_value_ se lay lam dai hien cho category list: =>>  "category_last_value_company_type",	
      ####  "category_last_value_id_management",	"category_last_value_id_office",	"category_last_value_job_role_encode_knn" ,"category_last_value_new_work_province_id"
  
    for col in [
                'numeric_mean_of_num_month_contract' ,
                'numeric_count_job',
                'numeric_count_job_withoutNan',
                'numeric_mean_out_nan_of_employee_lv', 
                 'numeric_mean_of_employee_lv', 
                'numeric_olds'
    ]:  
      ### wordAdd - companyType
      temp =  df.groupby(['category_last_value_new_work_province_id', 
                          'category_last_value_company_type'])[col].agg(['min', 'max', 'mean', 'median','var', percentile_25, percentile_75 ]).rename(
                          {
                              'min': 'agg_'+ col +'_'+ 'workAddress_companyTye' +'_min',
                              'max': 'agg_'+ col + '_'+ 'workAddress_companyTye' +'_max',
                              'mean': 'agg_'+ col + '_'+ 'workAddress_companyTye' +'_mean',
                              'median': 'agg_'+ col + '_'+ 'workAddress_companyTye' +'_median',
                              'var': 'agg_'+ col + '_'+ 'workAddress_companyTye' + '_var',
                              'percentile_25': 'agg_'+ col + '_'+ 'workAddress_companyTye' + '_P25',
                              'percentile_75' : 'agg_'+ col + '_'+ 'workAddress_companyTye' + '_P75',
                          }, axis = 1
                      )
      df = pd.merge(df,temp,on= ['category_last_value_new_work_province_id',
                          'category_last_value_company_type'] ,how='left')
      del temp
      ## companyType - id_management
      temp =  df.groupby(['category_last_value_company_type',
                          'category_last_value_id_management'])[col].agg(['min', 'max', 'mean', 'median','var', percentile_25, percentile_75 ]).rename(
                          {
                              'min': 'agg_'+ col +'_'+ 'companyTye_idManagement' +'_min',
                              'max': 'agg_'+ col + '_'+ 'companyTye_idManagement' +'_max',
                              'mean': 'agg_'+ col + '_'+ 'companyTye_idManagement' +'_mean',
                              'median': 'agg_'+ col + '_'+ 'companyTye_idManagement' +'_median',
                              'var': 'agg_'+ col + '_'+ 'companyTye_idManagement' + '_var',
                              'percentile_25': 'agg_'+ col + '_'+ 'companyTye_idManagement' + '_P25',
                              'percentile_75' : 'agg_'+ col + '_'+ 'companyTye_idManagement' + '_P75',
                          }, axis = 1
                      )
      df = pd.merge(df,temp,on= ['category_last_value_company_type',
                          'category_last_value_id_management'] ,how='left')
      del temp
      ## companyType - job_role
      temp =  df.groupby(['category_last_value_company_type',
                          'category_last_value_job_role_encode_knn'])[col].agg(['min', 'max', 'mean', 'median','var', percentile_25, percentile_75 ]).rename(
                          {
                              'min': 'agg_'+ col +'_'+ 'companyTye_jobRole' +'_min',
                              'max': 'agg_'+ col + '_'+ 'companyTye_jobRole' +'_max',
                              'mean': 'agg_'+ col + '_'+ 'companyTye_jobRole' +'_mean',
                              'median': 'agg_'+ col + '_'+ 'companyTye_jobRole' +'_median',
                              'var': 'agg_'+ col + '_'+ 'companyTye_jobRole' + '_var',
                              'percentile_25': 'agg_'+ col + '_'+ 'companyTye_jobRole' + '_P25',
                              'percentile_75' : 'agg_'+ col + '_'+ 'companyTye_jobRole' + '_P75',
                          }, axis = 1
                      )
      df = pd.merge(df,temp,on= ['category_last_value_company_type',
                          'category_last_value_job_role_encode_knn'] ,how='left')
      del temp
      ## companyType - job_role - id_management
      temp =  df.groupby(['category_last_value_company_type',
                          'category_last_value_job_role_encode_knn',
                          'category_last_value_id_management'])[col].agg(['min', 'max', 'mean', 'median','var', percentile_25, percentile_75 ]).rename(
                          {
                              'min': 'agg_'+ col +'_'+ 'companyTye_jobRole_idManagement' +'_min',
                              'max': 'agg_'+ col + '_'+ 'companyTye_jobRole_idManagement' +'_max',
                              'mean': 'agg_'+ col + '_'+ 'companyTye_jobRole_idManagement' +'_mean',
                              'median': 'agg_'+ col + '_'+ 'companyTye_jobRole_idManagement' +'_median',
                              'var': 'agg_'+ col + '_'+ 'companyTye_jobRole_idManagement' + '_var',
                              'percentile_25': 'agg_'+ col + '_'+ 'companyTye_jobRole_idManagement' + '_P25',
                              'percentile_75' : 'agg_'+ col + '_'+ 'companyTye_jobRole_idManagement' + '_P75',
                          }, axis = 1
                      )
      df = pd.merge(df,temp,on= ['category_last_value_company_type',
                          'category_last_value_job_role_encode_knn',
                          'category_last_value_id_management'] ,how='left')       

      del temp
    ## create bining features: olds: numeric_olds
    df['category_bining_olds'] = df['numeric_olds'].apply(lambda row: bining_olds(row))
    ## check HN - 24, TPHCM - 58, Da Nang - 15, HAI PHONG - 27, CAN THO - 13
    df['numeric_is_in_bigCity'] = df['category_max_fre_new_work_province_id'].apply(lambda x: 1 if (x in [24, 58, 15, 13, 27]) else 0)
    df['numeric_is_in_bigCity_upToDate'] = df['category_last_value_new_work_province_id'].apply(lambda x: 1 if (x in [24, 58, 15, 13, 27]) else 0)
    

    ## count features
    ## count encoding
    for col in [
        "category_last_value_new_work_province_id",
        "category_last_value_company_type",
        "category_last_value_id_management",
        "category_last_value_job_role_encode_knn",
 

        "category_max_fre_new_work_province_id",
        "category_max_fre_company_type",
        "category_max_fre_id_management",
        "category_max_fre_job_role_encode_knn"]:
        temp = df[col].value_counts().to_dict()
        df['numeric_ce_'+col ] = df[col].map(temp)
        del temp
    df.fillna(-999, inplace=True)
    return df
  

In [131]:
tf_idf_cols = [col for col in train.columns if 'job_role_TFIDF_' in col]
col_with_out_tf_idf_cols = [col for col in train.columns if col not in tf_idf_cols]
## remove cols won't be used
col_with_out_tf_idf_cols.remove('job_role_fillNan')
col_with_out_tf_idf_cols.remove('job_role_fillNan_sentence')
col_with_out_tf_idf_cols.remove('job_role_TFIDF')
col_with_out_tf_idf_cols.remove('label')

label_train = train['label'].values
len(tf_idf_cols), len(col_with_out_tf_idf_cols), len(train.columns)

(128, 16, 148)

In [134]:
%%time
fe_train_with_out_tfidf = features_engineering(train[col_with_out_tf_idf_cols])
fe_test_with_out_tfidf = features_engineering(test[col_with_out_tf_idf_cols])

C:\Users\Toan\AppData\Local\Temp/ipykernel_1020/20443307.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category_max_fre_' + col] = df[col].apply(lambda x: max_frequent(x) ) ## category
C:\Users\Toan\AppData\Local\Temp/ipykernel_1020/20443307.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['category_max_fre_out_filled_var_' + col] = df[col].apply(lambda x: max_frequent_without_filled_var(x) ) ## category
C:\Users\Toan\AppData\Local\Temp/ipykernel_1020/20443307.py:85: SettingWithCopyWarning

Wall time: 4min 41s


In [135]:
fe_train = pd.concat([fe_train_with_out_tfidf, train[tf_idf_cols], train['label'] ], axis = 1)
fe_test = pd.concat([fe_test_with_out_tfidf, test[tf_idf_cols] ], axis = 1)


In [136]:
numeric_ml_useful_cols = [col for col in fe_train.columns if 'numeric_' in col] + tf_idf_cols
category_ml_useful_cols = [col for col in fe_train.columns if (('category_' in col) and ('numeric_ce_' not in col) )]
old_cols = ["bithYear",	"gender","new_province_id", "id_bh", "label"]
# agg_cols = 
ml_useful_cols_train = numeric_ml_useful_cols + category_ml_useful_cols + old_cols
ml_useful_cols_test = numeric_ml_useful_cols + category_ml_useful_cols + old_cols[:-1]
len(ml_useful_cols_train), len(ml_useful_cols_test)

(418, 417)

In [137]:
len(set(ml_useful_cols_train))

418

## 6. Test baseline

In [138]:
category_cols = category_ml_useful_cols + [ "new_province_id"]

In [139]:
category_cols 

['category_max_fre_company_type',
 'category_max_fre_out_filled_var_company_type',
 'category_last_value_company_type',
 'category_max_fre_id_management',
 'category_max_fre_out_filled_var_id_management',
 'category_last_value_id_management',
 'category_max_fre_id_office',
 'category_max_fre_out_filled_var_id_office',
 'category_last_value_id_office',
 'category_max_fre_job_role_encode_knn',
 'category_max_fre_out_filled_var_job_role_encode_knn',
 'category_last_value_job_role_encode_knn',
 'category_max_fre_new_work_province_id',
 'category_max_fre_out_filled_var_new_work_province_id',
 'category_last_value_new_work_province_id',
 'category_bining_olds',
 'new_province_id']

In [140]:
df_train = fe_train[ml_useful_cols_train]
df_test = fe_test[ml_useful_cols_test]

In [141]:
df_train.shape, df_test.shape

((27502, 418), (18134, 417))

In [105]:
from lightgbm import LGBMClassifier as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [154]:
skf = StratifiedKFold(n_splits= 5, random_state= 42, shuffle= True)
X_train = df_train.drop(columns = ['label', 'id_bh']) 
X_train[category_cols] = X_train[category_cols].astype('category')
y_train = df_train['label']
params_k = {
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'num_class':7,
            'categorical_feature': category_cols,
            'verbose' : -1,
            "random_seed": 42,
            # "bagging_fraction": 0.7
}


In [155]:
X_train.shape

(27502, 416)

## Features selection

In [88]:
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from matplotlib.ticker import MaxNLocator

## CROSS VALIDATE

In [157]:
## Kfold
cv_score = []
training_score = []
feature_importance_df =  pd.DataFrame()
for n_fold, (train_idx, valid_idx) in enumerate(skf.split(X_train, y_train)):
    print("Fold %s" % (n_fold))
    train_x, train_y = X_train.iloc[train_idx], y_train.iloc[train_idx]
    valid_x, valid_y = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

    clf = lgb(
        **params_k
    )
    clf.fit(train_x, train_y, verbose= False)
    score_cv = f1_score(valid_y, clf.predict(valid_x),  average='macro')
    score_training = f1_score(train_y, clf.predict(train_x),  average='macro')

    # For create feature importances
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = X_train.columns
    fold_importance_df["importance"] = clf.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    print('F1_SCORE_CV: ', round(score_cv, 4), '\n')
    print('F1_SCORE_TRAINING: ', round(score_training, 4), '\n')

    cv_score.append(score_cv)
    training_score.append(score_training)

Fold 0


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


F1_SCORE_CV:  0.7872 

F1_SCORE_TRAINING:  1.0 

Fold 1


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


F1_SCORE_CV:  0.7471 

F1_SCORE_TRAINING:  1.0 

Fold 2


...

F1_SCORE_CV:  0.8122 

F1_SCORE_TRAINING:  1.0 

Fold 3


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


F1_SCORE_CV:  0.8012 

F1_SCORE_TRAINING:  1.0 

Fold 4


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


F1_SCORE_CV:  0.7898 

F1_SCORE_TRAINING:  1.0 



In [158]:
##  - 0.8049874090219371 
print("CV SCORE: {} - std: {}".format(sum(cv_score)/ 5,std(cv_score) ))
print("TRAINING SCORE: {} - std: {}".format(sum(training_score)/ 5,std(training_score) ))

CV SCORE: 0.7875084698209586 - std: 0.024681535781621092
TRAINING SCORE: 0.9999959189404791 - std: 9.125526508854801e-06


In [159]:
def display_importances(feature_importance_df_, top_k):

    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(
        by="importance", ascending=False)[:top_k].index
    
    # best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    
    # plt.figure(figsize=(12, 8))
    # sns.barplot(x="importance", y="feature", 
    #             data=best_features.sort_values(by="importance", ascending=False))
    # plt.title('LightGBM Features (avg over folds)')
    # plt.tight_layout()
    return cols
# top_fea_50 = display_importances(feature_importance_df_=feature_importance_df, top_k = 20)
# top_fea_100 = display_importances(feature_importance_df_=feature_importance_df, top_k = 100)
# top_fea_150 = display_importances(feature_importance_df_=feature_importance_df, top_k = 150)
top_fea_200 = display_importances(feature_importance_df_=feature_importance_df, top_k = 200)
top_fea_250 = display_importances(feature_importance_df_=feature_importance_df, top_k = 250)
top_fea_300 = display_importances(feature_importance_df_=feature_importance_df, top_k = 300)

In [161]:
## top 250 cv - 0.7978353846662485 - std: 0.018426308739477273 1st
## top 200 cv - 0.7937366078480538 - std: 0.021380086178773374
## top 150 cv - 0.7899814071413578 - std: 0.03008877391310114
## top 100 cv - 0.7802258450609931 - std: 0.02259313462464178
## top 300 cv - 0.7930353865893962 - std: 0.026677523879470658
top_fea_50_cols_list =  list(top_fea_250)
cv_score = []
training_score = []
X_train_useful_cols = X_train[top_fea_50_cols_list]
feature_importance_df =  pd.DataFrame()
for n_fold, (train_idx, valid_idx) in enumerate(skf.split(X_train_useful_cols, y_train)):
    print("Fold %s" % (n_fold))
    train_x, train_y = X_train_useful_cols.iloc[train_idx], y_train.iloc[train_idx]
    valid_x, valid_y = X_train_useful_cols.iloc[valid_idx], y_train.iloc[valid_idx]

    clf = lgb(
        **params_k
    )
    clf.fit(train_x, train_y, verbose= False)
    score_cv = f1_score(valid_y, clf.predict(valid_x),  average='macro')
    score_training = f1_score(train_y, clf.predict(train_x),  average='macro')
    print('F1_SCORE_CV: ', round(score_cv, 4), '\n')
    print('F1_SCORE_TRAINING: ', round(score_training, 4), '\n')

    cv_score.append(score_cv)
    training_score.append(score_training)
print("CV SCORE: {} - std: {}".format(sum(cv_score)/ 5,std(cv_score) ))
print("TRAINING SCORE: {} - std: {}".format(sum(training_score)/ 5,std(training_score) ))

Fold 0


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


F1_SCORE_CV:  0.7917 

F1_SCORE_TRAINING:  1.0 

Fold 1


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


F1_SCORE_CV:  0.776 

F1_SCORE_TRAINING:  1.0 

Fold 2


...

F1_SCORE_CV:  0.8233 

F1_SCORE_TRAINING:  1.0 

Fold 3


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


F1_SCORE_CV:  0.8089 

F1_SCORE_TRAINING:  1.0 

Fold 4


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


F1_SCORE_CV:  0.7893 

F1_SCORE_TRAINING:  1.0 

CV SCORE: 0.7978353846662485 - std: 0.018426308739477273
TRAINING SCORE: 0.9999836758985203 - std: 1.7072051926739718e-05


In [162]:
## select top 250 features important

## 7. Predict

### predict for SMOTE data

In [ ]:
# submit_file.head()
# # test_label.head()

In [ ]:
# submit_file.to_csv("data/submmit/baseline_lgb_addAggFeaPhase2.csv", index=False)

In [ ]:
# submit_file['label'].value_counts()

## 8. OPTUNA TUNE

In [163]:
selected_features = list(top_fea_250)
X_train = df_train.drop(columns = ['label', 'id_bh']) 
X_train[category_cols] = X_train[category_cols].astype('category')
X_train = X_train[selected_features]

In [164]:
import optuna
def objective(trial):
    # Specify a search space using distributions across plausible values of hyperparameters.
    param = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class':7,
        'categorical_feature': category_cols,       
        "seed": 42,
        # 'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        # 'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 12),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1200),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 1),

        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        # 'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        # 'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
    }
    
    # Run LightGBM for the hyperparameter values
    lgbcv = lgb(**param,
                #    categorical_feature=ids_of_categorical,              
                                  
                  )
    cv_score = []
    for n_fold, (train_idx, valid_idx) in enumerate(skf.split(X_train , y_train)):
        # print("Fold %s" % (n_fold))
        train_x, train_y = X_train .iloc[train_idx], y_train.iloc[train_idx]
        valid_x, valid_y = X_train .iloc[valid_idx], y_train.iloc[valid_idx]

        clf = lgbcv
        clf.fit(train_x, train_y, verbose= False)
        score = f1_score(valid_y, clf.predict(valid_x),  average='macro')
        print('F1_SCORE: ', round(score, 4), '\n')
        cv_score.append(score)
    
    return sum(cv_score)/len(cv_score)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50) 

[I 2022-04-18 22:06:30,172] A new study created in memory with name: no-name-fa7446fa-507c-44a1-98af-6489a65a4f5c
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.

[LightGBM] [Warning] feature_fraction is set=0.9727895077873797, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9727895077873797
[LightGBM] [Warning] bagging_fraction is set=0.5920311821910044, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5920311821910044
F1_SCORE:  0.4047 

[LightGBM] [Warning] feature_fraction is set=0.9727895077873797, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9727895077873797
[LightGBM] [Warning] bagging_fraction is set=0.5920311821910044, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5920311821910044


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


F1_SCORE:  0.3313 

[LightGBM] [Warning] feature_fraction is set=0.9727895077873797, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9727895077873797
[LightGBM] [Warning] bagging_fraction is set=0.5920311821910044, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5920311821910044


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


...

F1_SCORE:  0.8012 

[LightGBM] [Warning] feature_fraction is set=0.9508371613914272, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9508371613914272
[LightGBM] [Warning] bagging_fraction is set=0.6600856967220281, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6600856967220281


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


F1_SCORE:  0.7687 

[LightGBM] [Warning] feature_fraction is set=0.9508371613914272, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9508371613914272
[LightGBM] [Warning] bagging_fraction is set=0.6600856967220281, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6600856967220281


C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
[I 2022-04-18 23:45:47,969] Trial 49 finished with value: 0.7799246907255758 and parameters: {'num_leaves': 2, 

F1_SCORE:  0.7876 



In [166]:
## cv :0.8101751814855515
best_params = {
        'boosting_type': 'gbdt',
        'objective': 'multiclass',
        'num_class':7,
        'categorical_feature': category_cols,       
        "seed": 42,} 
best_params.update(study.best_params)
best_params

{'boosting_type': 'gbdt',
 'objective': 'multiclass',
 'num_class': 7,
 'categorical_feature': ['category_max_fre_company_type',
  'category_max_fre_out_filled_var_company_type',
  'category_last_value_company_type',
  'category_max_fre_id_management',
  'category_max_fre_out_filled_var_id_management',
  'category_last_value_id_management',
  'category_max_fre_id_office',
  'category_max_fre_out_filled_var_id_office',
  'category_last_value_id_office',
  'category_max_fre_job_role_encode_knn',
  'category_max_fre_out_filled_var_job_role_encode_knn',
  'category_last_value_job_role_encode_knn',
  'category_max_fre_new_work_province_id',
  'category_max_fre_out_filled_var_new_work_province_id',
  'category_last_value_new_work_province_id',
  'category_bining_olds',
  'new_province_id'],
 'seed': 42,
 'num_leaves': 12,
 'n_estimators': 606,
 'learning_rate': 0.04758825215802909,
 'feature_fraction': 0.9125911608337296,
 'bagging_fraction': 0.6099589912189116}

In [167]:
id_test = df_test['id_bh']
X_test = df_test.drop(columns = [ 'id_bh']) 
X_test[category_cols] = X_test[category_cols].astype('category')

X_test = X_test[selected_features]
clf = lgb(**best_params)
clf.fit( X_train, y_train)
preds =  clf.predict(X_test)

predict_df = pd.DataFrame()
predict_df['id_bh'] =id_test
predict_df['label'] = preds

## merge voi label_test
submit_file = concat_data(test_label,predict_df )

C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
C:\Users\Toan\anaconda3\envs\ml_com\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] feature_fraction is set=0.9125911608337296, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9125911608337296
[LightGBM] [Warning] bagging_fraction is set=0.6099589912189116, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6099589912189116


In [168]:
submit_file.to_csv("data/submmit/tuned_lg_with_top250_selected_features_cv081017.csv", index=False)

In [172]:
top_fea_250_df = pd.DataFrame(list(top_fea_250), columns = ['top_250_fea'])
top_fea_250_df.to_csv("top250fea.csv",  index = False)
